# Hock-Schittkowsky 71 - Degenerate
(Adapted from Problem number [71](https://www.coin-or.org/Ipopt/documentation/node20.html) from the Hock-Schittkowsky test suite and augmented with the formulation 4.2 of Curtis, et al. SIAM Journal on Optimization. 2009 Sep 16;20(3):1224-49.)

> **Requirements**
> - python3.X
> - pyomo 5.X.X
> - [ipopt](https://github.com/coin-or/Ipopt) 3.12.12 (tested with)

### 0. Introduction

In this problem, an ill-posed problem is tested with *ipopt*; to demonstrate the effects of "bad" modelling.
Firstly, consider the problem:

\begin{align}
\text{minimize}_{x} \quad & x_{1}x_{4} \left(x_{1} + x_{2} + x_{3} \right) + x_{3} \\
\text{subject to} \quad & x_{1}x_{2}x_{3}x_{4} \geq 25, \\
& x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} = 40 \\
& 1 \leq x_{1}, x_{2}, x_{3}, x_{4} \leq 5,
\end{align}

With initial guess $x_{IG} = \left(1,5,5,1 \right)$. This problem is well-posed in the sense that the gradients of the constraints at the initial guess and at the solution $x^{*} = \left(1, 4.7443, 3.8211, 1.3794\right)$ are linearly independent.  
However, one can make a modelling mistakes and create problematic cases; particularly for solvers like *ipopt*.
For example, suppose that a constraint is added to the previous model so that:

\begin{align}
c_{1} \left(x\right) &= 0 ,\\
c_{1} \left(x \right) - c_{1}^{2} \left( x \right) & = 0,
\end{align}

where the last constraint contains information of the already defined $c_{1} \left(x \right)= x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} - 40 = 0$. Clearly, this constraint does not change the feasible set of the original problem. However, the linearized functions of these constraints are inconsistents at all points for which $c_{1} \neq 0$.

The purpose of this exercise is to notice the behaviour of the solver as a result of bad modelling.

### 1. Package imports and model declaration

For this model, the use of an *AbstractModel* will be shown. This kind of model is typical for problems whose structure depend on data. The *AbstractModel* will act as a blueprint for the overall model but it can *only* behave as a regular model after its instantiation. Note that for this case study a *ConcreteModel* would work just as fine.

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
m = AbstractModel()  #: Abstract model for educational purposes

### 2. Indexing sets and Variables

The problem HK071 has four variables $x_{1}, x_{2}, x_{3}, x_{4}$. It is also possible to create a vector $x \in \mathbb{R}^{4}$. 
In abstract models, the sets and parameter data can be passed from a [text file](https://ampl.com/BOOK/CHAPTERS/12-data.pdf), the data can be also be written in tabular form.  
It is known that the set of variables goes from 1...4, however this information can be part of the data file of the abstract model.

In [2]:
m.i = Set()  #: No particular initialization here.

In [3]:
x_guess = {1: 1, 2: 5, 3: 5, 4:1}

The variables' initial value can be declared within the data file as well. Since the bounds of the variables are the same, it is possible to use the `bounds` keyword to assign the same bounds for all the variables.

In [4]:
#m.x = Var(m.i, initialize=x_guess, bounds=(1,5))
m.x = Var(m.i, bounds=(1,5))

### 3. Constraints and Objective

In [5]:
def _c0_rule(mod):
    return mod.x[1] * mod.x[2] * mod.x[3] * mod.x[4] >= 25
m.c0 = Constraint(rule=_c0_rule)  #: nothing special about this one

As noted in other examples meaningful *Constraint* objects require an expression. Moreover, there exists *Expression* objects that encapsulate symbolic relationships between variables. *Expression* objects can be conviniently used, for example, to initialize parts of a *Constraint* object. In the following example an *Expression* object is used to set the right-hand-side of the second constraint of the model.

In [6]:
m.e1 = Expression(rule=lambda mod: mod.x[1]**2 + mod.x[2]**2 + mod.x[3]**2 + mod.x[4]**2 - 40)  #: the expression is declared

Then the expression is passed to the constraint objects.

In [7]:
m.c1 = Constraint(expr=m.e1 == 0)
m.c2 = Constraint(expr=m.e1 - m.e1 ** 2 == 0)

Finally, the objective is also defined using a python function:

In [8]:
def _objective_rule(mod):
    return mod.x[1] * mod.x[1] * (mod.x[1] + mod.x[2] + mod.x[3]) + mod.x[3]
m.of = Objective(rule=_objective_rule, sense=minimize)

Note that for the *AbstractModel* class, the constraints require python functions to pass on the respective expressions using the `rule` keyword.

### 4. Solution and discussion

In [9]:
with open("hk071.dat", "w") as f:
    f.write("set i := 1 2 3 4;\n")
    f.close()

Before solving the problem an instance of the model must be constructed. If there is a data file, the construction of the instance can be done using the `create_instance` method as follows,

In [10]:
instance = m.create_instance("hk071.dat")

At this point the instance of the model has been created. The instance basically behaves like a `ConcreteModel` and can be modified if necesary. For example, the variables can be set to the initial guess using the `set_value` method

In [11]:
instance.x[1].set_value(1)
instance.x[2].set_value(5)
instance.x[3].set_value(5)
instance.x[4].set_value(1)

Then the solver can be created and called to solve the problem.

In [12]:
opt = SolverFactory('ipopt')
try:
    res = opt.solve(instance, tee=True)
except ValueError:
    print("Warning, something went wrong!")

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Tot

If *MUMPS* is used as linear solver and the default options are used, the solution will fail. In this case, the default output of ipopt reveals that the column *inf_du* has grown significantly right before termination. It is likely related to the failure to satisfy the *constraint qualifications* given the current 
formulation of the problem.  
It is clear that a slight mistake in the modelling can be problematic. Nevertheless, for this case trying a different starting point or different options *might* actually help solving the problem.

### Optional: Fix the model

The model can be easily fixed if the last constraint is removed. This can be done by using the `deactivate` method.

In [13]:
instance.c2.deactivate()

And then solve again.

In [14]:
res = opt.solve(instance, tee=True)

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Tot

In [15]:
instance.x.pprint()

x : Size=4, Index=i
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      1 :     1 :                1.0 :     5 : False : False :  Reals
      2 :     1 : 3.4494896699659057 :     5 : False : False :  Reals
      3 :     1 :  1.449489760695073 :     5 : False : False :  Reals
      4 :     1 :                5.0 :     5 : False : False :  Reals
